In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("ticketSearchResults-2015-02-09-20_48_16.txt", sep="~")
df = df.fillna("NULL")
df

,Ticket,Call Dt/Tm,Header,County,City,Excav Company,Address,Street,Inter1,Inter2
0,100600001,2010-03-01 02:51:21-05,EMERGENCY,BERGEN,ROCKLEIGH,ORANGE AND ROCKLAND UTILITIES,9,PIERMONT,HARING FARM,ROCKLEIGH
1,100600002,2010-03-01 03:00:33-05,EMERGENCY,BERGEN,ALPINE,ORANGE AND ROCKLAND UTILITIES,441,US HWY 9W,ORBACH,NULL
2,100600003,2010-03-01 04:05:36-05,EMERGENCY,MONMOUTH,HAZLET,VERIZON,NULL,LAUREL,HELFRICH,NULL
3,100600004,2010-03-01 05:15:23-05,EMERGENCY,ESSEX,FAIRFIELD,PSE&G,11,MADISON,PLYMOUTH,NULL
4,100600005,2010-03-01 05:21:24-05,EMERGENCY,PASSAIC,HALEDON,MONTANA CONSTRUCTION,89,LEE,LEGION,HENRY
...,...,...,...,...,...,...,...,...,...,...
46898,100902145,2010-03-31 23:21:10-04,ROUTINE,GLOUCESTER,WASHINGTON,STEPHENS EXTERMINATING,1701,HAWTHORNE,ARBOUR,NULL
46899,100902146,2010-03-31 23:26:30-04,ROUTINE,GLOUCESTER,WASHINGTON,STEPHENS EXTERMINATING,401,BRIARWOOD,ARBOUR,NULL
46900,100902147,2010-03-31 23:29:39-04,ROUTINE,MIDDLESEX,PLAINSBORO,TELECABLE,101,MORGAN,ENTERPRISE,NULL
46901,100902148,2010-03-31 23:37:03-04,ROUTINE,MONMOUTH,SHREWSBURY BORO,TELECABLE,149,AVENUE OF THE CMN,BROAD,NULL


In [19]:
file = open('test.sql', 'w')
for i in df.index:
    file.write("INSERT INTO geocd_nj (ticket, dt, tm, hder, county, excav_comp, inter1, inter2) VALUES(%d, '%s', '%s', '%s', '%s', '%s', '%s', '%s');\n"
         % (df.loc[i, 'Ticket'],
            df.loc[i, 'Call Dt/Tm'].split(' ')[0],
            df.loc[i, 'Call Dt/Tm'].split(' ')[1],
            df.loc[i, 'Header'],
            df.loc[i, 'County'],
            df.loc[i, 'Excav Company'].replace("'", "''"),
            df.loc[i, 'Inter1'].replace("'", "''"),
            df.loc[i, 'Inter2'].replace("'", "''")))
    file.write("UPDATE geocd_nj SET rating = g.rating, lon = ST_X(g.geomout),lat = ST_Y(g.geomout), st_no = (addy).address,st_nm = (addy).streetname, city = (addy).location,state_nm = (addy).stateabbrev,zip = (addy).zip FROM geocode('%s %s, %s, NJ', 1) As g WHERE ticket = %d;\n"
         % (df.loc[i, 'Address'],
            df.loc[i, 'Street'].replace("'", "''"),
            df.loc[i, 'City'],
            df.loc[i, 'Ticket']))
file.close()